In [ ]:
# Include all the utilities
include("utilities.jl");

In [ ]:
# Read mazes that should be solved
max_size = 10
n_averages = 100

mazes = [zeros(2*i+1, 2*i+1) for i in 1:max_size, j in 1:n_averages];

for i in 1:max_size
    test = readdlm("mazes/mazes$i.csv", ',', Float64)
    test = reshape(test, 100, 2*i+1, 2*i+1)
    for j in 1:n_averages
        mazes[i,j] = test[j,:,:]
    end
end

In [ ]:
# Allocate memory for the computations with fixed discount
timesBCP = zeros(max_size, n_averages);
rewardsBCP = zeros(max_size, n_averages);
statusBCP = [];

In [ ]:
# Define the visibility / observability mask
V = [1 1 1; 1 0 1; 1 1 1];

In [ ]:
# Fix discount factor
γ = 0.9999;

In [ ]:
# Do the computations
@elapsed for i in 1:max_size
    for j in 1:n_averages
        M = mazes[i,j];
        states = listOfStates(M);
        goal = rand(states);
        α = transitionKernel(M,A,goal);
        β = observationKernel(M,V);
        r = instReward(M,A,goal);
        μ = initialDistribution(M);
        # Evaluate BCP
        πBCP, modelBCP = BCP(α, β, r, μ, γ, 0)
        rewardsBCP[i,j] = R(πBCP, α, β, γ, μ, r)
        timesBCP[i,j] = solve_time(modelBCP)
        statusBCP = vcat(statusBCP, raw_status(modelBCP))
    end
    println("BCP time for size $i: ", sum(timesBCP[i,:]) / n_averages)
    println("BCP reward for size $i: ", sum(rewardsBCP[i,:]) / n_averages)
end

In [ ]:
writedlm("data/BCPDiscountFixedRewardsSmall.csv", rewardsBCP, ',')
writedlm("data/BCPDiscountFixedTimesSmall.csv", timesBCP, ',')
writedlm("data/BCPDiscountFixedStatusSmall.csv", statusBCP, ',')